In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

# Load the data

In [ ]:
import datetime
from helpers import *

height, weight, gender = load_data(sub_sample=False, add_outlier=False)
x, mean_x, std_x = standardize(height)
y, tx = build_model_data(x, weight)

In [ ]:
y.shape, tx.shape

# 1 Computing the Cost Function

Answers to questions:
    1. What does each column of X represent?
       The first column is the "extra column" for the bias (constant) term.
       The second column stores the single scalar value that is each data point (the heights of people).
       Each column is a different feature of the data.
    2. What does each row of X represent?
       Each row of X is an entire data point.
    3. The 1s in the first column represent the bias term. By adding this column, 
       we make the code and mathematical representations cleaner.
    4. size(y) = 3, size(X) = (3,2), X(3,2) is the height of the person no. 3.
    

Fill in the `compute_loss` function below:
<a id='compute_loss'></a>


In [ ]:
def compute_loss(y, tx, w):
    """Calculate the loss.

    You can calculate the loss using mse or mae.
    """
    N = len(y)
    e = y - (tx @ w)
    L = (0.5 / N) * np.dot(e, e)
    return L

# 2 Grid Search

Fill in the function `grid_search()` below:

In [ ]:
def grid_search(y, tx, w0, w1):
    """Algorithm for grid search."""
    losses = np.zeros((len(w0), len(w1)))
    
    for i, t0 in enumerate(w0):
        for j, t1 in enumerate(w1):
            losses[i, j] = compute_loss(y, tx, (t0, t1))
    
    return losses

Let us play with the grid search demo now!

In [ ]:
from grid_search import generate_w, get_best_parameters
from plots import grid_visualization

# Generate the grid of parameters to be swept
grid_w0, grid_w1 = generate_w(num_intervals=100)

# Start the grid search
start_time = datetime.datetime.now()
grid_losses = grid_search(y, tx, grid_w0, grid_w1)

# Select the best combinaison
loss_star, w0_star, w1_star = get_best_parameters(grid_w0, grid_w1, grid_losses)
end_time = datetime.datetime.now()
execution_time = (end_time - start_time).total_seconds()

# Print the results
print("Grid Search: loss*={l}, w0*={w0}, w1*={w1}, execution time={t:.3f} seconds".format(
      l=loss_star, w0=w0_star, w1=w1_star, t=execution_time))

# Plot the results
fig = grid_visualization(grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight)
fig.set_size_inches(10.0,6.0)
fig.savefig("grid_plot")  # Optional saving

b) 
This is does not look like a good estimate. The plot of the fit on the right shows that the model line is not very well centered on the data points, as we would expect of a true best-fit line. This is because, as we can see graphically on the plot on the left, the w* found by grid search is not the global minimum of the loss function. The MSE plot is not smooth because our grid is too coarse (only 10 samples in each dimension).

After changing the grid-search to use 50 samples in each dimension, the fit looks a lot closer to a best-fit line (it is more centered on the data). This is because the search found a point closer to the global minimum of the loss function, as seen on the left.

c) 
1. In order to obtain a good fit, we need a relatively fine grid. Only then can we be sure that we are somewhat close to the global minimum.
2. While the finer the grid does yield better results, it also greatly increases the runtime. 
3. The runtime scales roughly with the square of the number of grid points, so twice the grid points takes 4 times as long.

# 3 Gradient Descent

Again, please fill in the functions `compute_gradient` below:

In [ ]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    N = len(y)
    e = y - (tx @ w)
    
    return - (1.0/N) * np.transpose(tx) @ e

b)
Given the shape of the MSE plot (a paraboloid), the magnitude of the gradient provides an estimate of how far we are from the minimum.
In 1D, the absolute value of the derivative of a parabola increases linearly away from the extrema of the parabola. So the further you are from the minimum, the "steeper" the curve is and hence the larger the magnitude of the derivative. The same happens in 2D, where the magnitude of the gradient increases with the distance from the extrema of the paraboloid.

This also means that the gradient descent algorithm will take larger steps if it is far away from the minimum, and the step size will decrease as we get closer to it.

In [ ]:
compute_gradient(y, tx, np.array([100,20])), compute_gradient(y, tx, np.array([20,10]))

Please fill in the functions `gradient_descent` below:

In [ ]:
def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and loss
        grad = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        
        # update w by gradient
        w = w - gamma*grad
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

Test your gradient descent function through gradient descent demo shown below:

In [ ]:
# from gradient_descent import *
from plots import gradient_descent_visualization

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.1

# Initialization
w_initial = np.array([-10, 10])

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = gradient_descent(y, tx, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

# Time Visualization
from ipywidgets import IntSlider, interact

def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        gradient_losses, gradient_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_ws)))

c)
- Yes, the cost is being minimized. As we can see, the loss decreases with each iteration.
- Yes, the algorithm is converging. In particular, the loss hardly changes after the 8th iteration.
- The final model parameters seem to be good, they appear to be close to the global minimum. Moreover, the fit looks close to the expected best fit line.

d)
The algorithm did not converge for gamma = [0.001, 0.01, 2, 2,5], and it did for [0.05, 1]. Moreover, it converged quickly for g=1 (2 iterations), and more slowly for g=0.5 (14 iterations). For small gamma, the algorithm made very little progress toward the minimum. For large gamma, it overshot the minimum and was never able to find it.

Trying different initializations shows that gradient descent is more robust to the choice of initial guess. The algorithm converge for values somewhat close to the minimum, and was well on its way for values far away (but just didn't have enough iterations to get there).

# 4 Stochastic gradient descent

In [ ]:
def compute_stoch_gradient(y, tx, w, batch_size):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    for mb_y, mb_tx in batch_iter(y, tx, batch_size):
        grad = compute_gradient(mb_y, mb_tx, w)
    return grad
  
def stochastic_gradient_descent(
        y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic gradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        
        # compute gradient and loss
        grad = compute_stoch_gradient(y, tx, w, batch_size)
        loss = compute_loss(y, tx, w)
        
        # update w by gradient
        w = w - gamma*grad
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    
    return losses, ws

In [ ]:
# from stochastic_gradient_descent import *

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.1
batch_size = 1

# Initialization
w_initial = np.array([0, 0])

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = stochastic_gradient_descent(
    y, tx, w_initial, batch_size, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        sgd_losses, sgd_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gradient_ws)))

# 5 Effect of Outliers

### Exercise 5


In [ ]:
# Reload sub-sample of data
from helpers import *
height, weight, gender = load_data(sub_sample=True, add_outlier=True)
x, mean_x, std_x = standardize(height)
y, tx = build_model_data(x, weight)

#######################################################################################
# plot data
#######################################################################################
from plots import gradient_descent_visualization

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.2

# Initialization
w_initial = np.array([100, 10])

# Start gradient descent.
start_time = datetime.datetime.now()
gd_losses, gd_ws = gradient_descent(y, tx, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        gd_losses, gd_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(gd_ws)))

Just by adding a few outliers, the model looks a lot worse, even though the gradient descent still finds a minimum of the MSE.

# 6 Subgradient Descent

### Exercise 6

Modify the function `compute_loss(y, tx, w)` for the Mean Absolute Error cost function [here](#compute_loss)

In [ ]:
# Sources:
# - http://www.princeton.edu/~yc5/ele522_optimization/lectures/subgradient_methods.pdf
# - https://en.wikipedia.org/wiki/Subgradient_method
# - https://en.wikipedia.org/wiki/Subderivative

def compute_loss(y, tx, w):
    """Calculate the loss.

    You can calculate the loss using mae.
    """
    N = len(y)
    e = y - (tx @ w)
    # The MAE is just the L1 norm of the error vector scaled by 1/N
    L = (1.0 / N) * np.linalg.norm(e, 1)
    return L

def compute_subgradient(y, tx, w):
    """Compute the sub-gradient.
        At points where the loss function is continuous, the gradient is just the row sum of D*X
        where D is a diagonal matrix such that d_{ii} is the sign of the i-th entry of e=y-Xw
    """
    N = len(y)
    e = y - (tx @ w)
    sign = np.sign(e)
    if any(sign == 0):
        print("DISCONTINUITY!")
        
    sign = np.where(sign == 0, 1, sign)
    D = np.diag(sign)
    sgrad = - np.sum(D @ tx, axis=0)
    
    return sgrad

def subgradient_descent(y, tx, initial_w, max_iters, gamma):
    """Subgradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and loss
        grad = compute_subgradient(y, tx, w)
        loss = compute_loss(y, tx, w)
        
        # update w by gradient
        w = w - gamma*grad
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Subgradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

#######################################################################################
# plot data
#######################################################################################
# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.05

# Initialization
w_initial = np.array([0, 0])

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = subgradient_descent(
    y, tx, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        sgd_losses, sgd_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(sgd_ws)))

b)
Subgradient descent with MAE yields a model that is close to MSE without outliers! (but I had to reduce the learning rate)

MAE is also more resilient to outliers, as it converges to what looks like a best-fit model that basically ignores the outliers.

The SGD did not encounter any discontinuities. These are likely to be fairly rare since they have measure zero relative to the size of the whole parameter space (e.g. a finite set of points relative to the real line).

In [ ]:
def compute_stoch_subgradient(y, tx, w, batch_size):
    """Compute a stochastic gradient from just few examples n and their corresponding y_n labels."""
    for mb_y, mb_tx in batch_iter(y, tx, batch_size):
        grad = compute_subgradient(mb_y, mb_tx, w)
    return grad
  
def stochastic_subgradient_descent(
        y, tx, initial_w, batch_size, max_iters, gamma):
    """Stochastic subgradient descent algorithm."""
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # compute gradient and loss
        grad = compute_stoch_subgradient(y, tx, w, batch_size)
        loss = compute_loss(y, tx, w)
        
        # update w by gradient
        w = w - gamma*grad
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))
    
    return losses, ws

#######################################################################################
# plot data
#######################################################################################
# Define the parameters of the algorithm.
max_iters = 50
batch_size = 1
gamma = 2

# Initialization
w_initial = np.array([0, 0])

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = stochastic_subgradient_descent(
    y, tx, w_initial, batch_size, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

# Time Visualization
from ipywidgets import IntSlider, interact
def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        sgd_losses, sgd_ws, grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight, n_iter)
    fig.set_size_inches(10.0, 6.0)

interact(plot_figure, n_iter=IntSlider(min=1, max=len(sgd_ws)))

c)
The SGD algorithm appear to be more stable and not "jump around" the minimum as with MAE as compared to MSE. It might just be that MAE remains stable with higher learning rates than MSE.

Also SGD seems to make MSE more resilient to outliers on average, as the outliers are unlikely to get picked for the gradient.